In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import attr
import types
##
from util import tools
plt.rcParams['figure.figsize'] = (16, 4.5)
plt.style.use('seaborn-whitegrid')

In [7]:
class HJM_Gaussian:

    @staticmethod
    def calibrate_volatility(factors,data):
        diff_r = data.diff()[1:]
        sigma = diff_r.cov()
        sigma *= 252 # annualized
        eigval, eigvec = np.linalg.eig(sigma)
        # Make a list of (eigenvalue, eigenvector) tuples
        eig_pairs = [(np.abs(eigval[i]), eigvec[:,i]) for i in range(len(eigval))]
        eig_pairs.sort(key=lambda x: x[0], reverse=True)
        princ_eigvector = [eig_pairs[i][1] for i in range(0,factors)]
        princ_eigvalues = [eig_pairs[i][0] for i in range(0,factors)]
        W_matrix = np.vstack(princ_eigvector).T 
        Y = data.dot(W_matrix)
        vol_compressed = np.sqrt(princ_eigvalues)*W_matrix 
        return vol_compressed, eigval, eigvec

    def __init__(self,path,nfactors):
        self.data = tools.load_BTP_curve(path)       
        self.tenors = np.array(self.data.columns.values)
        self.vols, self._eigval, self._eigvec = HJM_Gaussian.calibrate_volatility(nfactors,self.data)
        self.nfactors = nfactors
        self._vol_functions = self._interp_volatility()[2]
        
    def PCA_results(self):
        tot = sum(self._eigval) # la somma degli autovettori è la varianza totale
        var_exp = [(i / tot)*100 for i in sorted(self._eigval, reverse=True)]
        cum_var_explained = np.cumsum(var_exp)
        print("Variance% explained by the first " +str(self.nfactors) +" eigenvectors:")
        print(str(np.round(cum_var_explained[self.nfactors],3)) + "%")
        [plt.bar(i +1, var_exp[i], alpha = 0.5, label = ("Component " + str(i +1))) for i in range(0,5)]
        [plt.step(i+1, cum_var_explained[i]) for i in range(0,5)]
        plt.title("Explained variance by components")
        plt.legend()
        plt.show()
        
    def _interp_volatility(self):
        """
        Parameters
        -------
        vols: volatility matrix to be interpolated (nObs x nFactors)
        tenors: array of tenors used for fitting
        Returns
        -------
        x: Volatility fitted polynomials
        save_pmts: poly weigths
        """
        x = np.zeros((self.vols.shape[0], self.vols.shape[1]))
        degree = 2
        save_pmts = []
        for i in range(0, self.vols.shape[1]):
            vol = np.array(self.vols[:,i].flatten())
            fit_vol = np.polyfit(x = self.tenors, y = vol, deg = degree)
            x[:,i] = np.polyval(fit_vol, self.tenors)
            degree = 4
            save_pmts.append(fit_vol)
        vol_functions = [np.poly1d(coeff) for coeff in save_pmts]
        return x, save_pmts, vol_functions
    
    def plotVolatility(self):
        fitted_vol, rg = self._interp_volatility()
        tenors = self.tenors
        plt.subplot(1, 3, 1), plt.plot(tenors, fitted_vol[:,0]), plt.plot(tenors, self.vols[:, 0])
        plt.legend(["Fitted Vol", "PC1 Vol"])
        plt.subplot(1, 3, 2), plt.plot(tenors, fitted_vol[:,1]), plt.plot(tenors, self.vols[:, 1])
        plt.legend(["Fitted Vol", "PC2 Vol"])
        plt.subplot(1, 3, 3), plt.plot(tenors, fitted_vol[:,2]), plt.plot(tenors, self.vols[:, 2])
        plt.legend(["Fitted Vol", "PC3 Vol"])
        plt.show()
        
    def _get_HJMdrift(self, T):
        mean = 0
        for sigma in self.vol_functions:
            comp_mean = integrate.quad(sigma, 0, T)[0] * sigma(T)
            mean += comp_mean
        return mean
    
    def monte_carlo_path(self):
        spot_BTPcurve = data[-1::].values.flatten()
        time_grid = np.linspace(0,5,500)
        np.random.seed(12)

In [6]:
path = './util/datastore/yield_italy_daily_2010_2020.xlsx'
factors = 3
hjm_class = HJM_Gaussian(path, factors)